In [ ]:
import tensorflow as tf
from google.colab import auth
from sklearn.model_selection import train_test_split
from google.auth import default

import pandas as pd
import gspread
import numpy as np
import time

##Build training set

In [ ]:
auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open("AI_data2.0").sheet1
# rows = pd.DataFrame.from_records(worksheet.get_all_records())
data_base = np.array(worksheet.get_all_values()[1:])
data_base

array([['1', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ...,
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0']], dtype='<U1')

In [ ]:
inputs = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

input_indexes = np.array([
    0, 0, 0, 0
])
outputs = np.empty((0, 19))

i = 0
while i < len(inputs):
  input = inputs[i]
  output = [0] * 19
  for row in data_base:
    output_indexes = []
    output_type = 0
    # figure out if the row contains the input
    for j in range(len(input)):
      value = int(row[j]) - input[j]
      if value > 0:
        output_type = 1
        output_indexes.append(j)
      elif value < 0:
        output_type = -1
        break

    if output_type == 1:
      for item in output_indexes:
        output[item - 4] = 1 #don't include the base flavors in the output
    if output_type == 0:
      output[18] = 1 #this represents the "end drink" value
  outputs = np.vstack((outputs, output))

  for k in range(input_indexes[i], len(output)-1): #don't include the end drink value
    new_input = np.copy(input)
    if output[k] > 0:
      new_input[k + 4] = input[k + 4] + 1
      inputs = np.vstack((inputs, new_input))
      input_indexes = np.append(input_indexes, k)
  # outputs.append(output)
  i += 1

In [ ]:
inputs = np.divide(inputs, 3)
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2)

## Train Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(88, activation="relu"),
    tf.keras.layers.Dense(44, activation="relu"),
    tf.keras.layers.Dense(19, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=["mae"])

model.fit(inputs_train, outputs_train, epochs=100)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5959 - mae: 0.4274
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3412 - mae: 0.1831 
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3480 - mae: 0.2130 
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3219 - mae: 0.1971 
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3327 - mae: 0.1987 
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2996 - mae: 0.1890 
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2909 - mae: 0.1811 
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2839 - mae: 0.1732 
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2620 - mae: 0.1618 
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2585 - mae: 0.1647 
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2523 - mae: 0.1614 
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2363 - mae: 0.1448 
Epoch 13/100
14/14 ━━━━━━━

In [ ]:
model.evaluate(inputs_test, outputs_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2211 - mae: 0.0821  


[0.238493412733078, 0.08730744570493698]

## Test

In [ ]:
def list_options(prediction):
  options = np.array(worksheet.get_all_values()[0])
  i = 0
  for item in prediction[0]:
    if item >= 0.4:
      if i == 18:
        print("finish drink")
      else:
        print(options[i + 4], " ", i+ 4)
    i += 1

In [ ]:
my_drink = np.array([[0, 0, 0.33, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
prediction = model.predict(my_drink)
list_options(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
finish drink


## Save Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmp0x_zupmn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 22), dtype=tf.float32, name='keras_tensor_44')
Output Type:
  TensorSpec(shape=(None, 19), dtype=tf.float32, name=None)
Captures:
  139893613833984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139893613974752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139893613832400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139893668284016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139893613974048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139893613982320: TensorSpec(shape=(), dtype=tf.resource, name=None)
